In [1]:
import sys
import os
from pathlib import Path
# This appends the directory one level up (the root of your project) to the sys.path.
# Modify the path depending on the location of modules you want to import.
sys.path.append(os.path.abspath('../../'))

from config.config_managers import DashboardConfigManager
from dataManager import DataManager
from dash import Dash
import pandas as pd
import plotly.express as px
from abc import ABC, abstractmethod
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

2024-12-28 02:08:47 - INFO - PyTorch version 2.2.2 available.


In [49]:
class Visualization(ABC):
    def __init__(self, data, mappings):
        self.data = data
        self.tag_mapping = mappings['tag_mapping']
        self.dataset_mapping = mappings['dataset_mapping']

    @abstractmethod
    def prepare_data(self):
        pass

    @abstractmethod
    def visualize(self):
        pass

    def replace_mappings(self, data):
        data = self.replace_tags(data)
        data = self.replace_model(data)
        # data['Model'] = data['Model'].replace(self.dataset_mapping)
        return data
    
    def replace_tags(self, data):
        data['Tag'] = data['Tag'].replace(self.tag_mapping)
        return data
    
    def replace_model(self, data):
        data['Model'] = data['Model'].replace(self.dataset_mapping)
        return data
    
    def replace_columns_and_index(self, data):
        return data.rename(columns=self.tag_mapping, index=self.tag_mapping)

        
    
    

mappings = {
    'tag_mapping': {'B-PERS': 'B-PER', 'I-PERS': 'I-PER',},
    'dataset_mapping': {'ANERCorp_CamelLab_arabertv02': 'AraBERTv02', 'conll2003_bert': 'BERT'}
}

class ReportBarChart(Visualization):
    def prepare_data(self):
        report_data = []
        for model_name, data_content in self.data.items():
            token_report = data_content.token_report
            token_report['Model'] = model_name
            report_data.append(pd.concat([
                token_report, 
            ]))
        report_df = pd.concat(report_data)
        report_df = report_df[~report_df['Tag'].isin(['accuracy/micro', 'macro', 'weighted'])]    
        
        return self.replace_mappings(report_df)

    def visualize(self):
        token_report_data = self.prepare_data()
        melted_df = token_report_data.melt(
        id_vars=["Tag", "Support", "Model"], 
        value_vars=["Precision", "Recall"], 
        var_name="Metric", 
        value_name="Value"
        )
        
        # Round the values for better readability
        melted_df['Value'] = melted_df['Value'].round(3)
        
        # Create the bar chart using Plotly Express
        fig = px.bar(
            melted_df, 
            x="Tag", 
            y="Value", 
            color="Metric",
            facet_row="Model",  # Separate rows for each model
            title="Breakdown of Precision and Recall Scores by Entity Tag, Categorized by Mode",
            labels={"Value": "Score", 'Tag': 'Entity'},
            barmode="group",
            template="plotly_white",
            facet_row_spacing=0.1,
            text='Value'  # Display the Value on top of each bar
        )
        fig.update_layout(
            width=1200,
            height=400,
            # margin=dict(t=50, l=50, r=50, b=50)
        )
        
        # Show the figure
        fig.show()
    
    def visualize_f1(self):
        token_report_data = self.prepare_data()
        token_report_data['F1'] = token_report_data['F1'].round(3)
        fig = px.bar(token_report_data,
                    x="Tag",
                    y="F1",
                    color="Model",
                    title="Breakdown of F1 Score Per Model",
                    barmode='group',
                    template="plotly_white",
                    text='F1',  # Display the actual Count on top of each bar
                    )
        
        fig.update_layout(
            width=1200,
            height=400,
            # margin=dict(t=50, l=50, r=50, b=50)
        )
                
        fig.show()
        
        
    def visualize_support(self):
        token_report_data = self.prepare_data()
        token_report_data = token_report_data[token_report_data['Tag'] !='O']
        fig = px.bar(token_report_data,
                    x="Tag", 
                    y="Support",
                    color="Model",
                    title="Breakdown of the Number of Examples Per Model",
                    barmode='group',
                    template="plotly_white",
                    # facet_row_spacing=0.1,  # Adjusted spacing
                    facet_col_spacing=0.08,
                    text='Support',  # Display the actual Count on top of each bar
                    )
        fig.update_layout(
            width=1200,
            height=400,
            # margin=dict(t=50, l=50, r=50, b=50)
        )
                
        fig.show()

class ConfusionBarChart(Visualization):
    def prepare_data(self):
        matrices = []
        for model_name, data_content in self.data.items():
            token_matrix = pd.DataFrame(data_content.token_confusion_matrix['confusion_matrix']).T 
            token_matrix['Model'] = model_name
            matrices.append(pd.concat([
                token_matrix, 
            ]))
        matrix_df = pd.concat(matrices)
        matrix_df.reset_index(inplace=True)
        matrix_df.rename(columns={'index': 'Tag'}, inplace=True)
        matrix_data = self.replace_mappings(matrix_df)
        
                
        grouped = matrix_data.groupby(['Tag', 'Model']).sum()
        grouped['Total'] = grouped['TP'] + grouped['FP'] + grouped['FN']
        
        matrix_data = matrix_data.merge(grouped['Total'], on=['Tag', 'Model'], how='left')
        
        matrix_data['TP_Count'] = matrix_data['TP']
        matrix_data['FP_Count'] = matrix_data['FP']
        matrix_data['FN_Count'] = matrix_data['FN']
        
        matrix_data['TP'] = matrix_data['TP'] / matrix_data['Total']
        matrix_data['FP'] = matrix_data['FP'] / matrix_data['Total']
        matrix_data['FN'] = matrix_data['FN'] / matrix_data['Total']
        
        confusion_scaled_df = matrix_data.melt(id_vars=["Tag", "Model"], value_vars=["TP", "FP", "FN"], var_name="Metric", value_name="Scale")
        confusion_count_df = matrix_data.melt(id_vars=["Tag", "Model"], value_vars=["TP_Count", "FP_Count", "FN_Count"], var_name="Metric", value_name="Count")
        confusion_count_df['Metric'] = confusion_count_df['Metric'].str.replace('_Count', '')
        confusion_data = confusion_scaled_df.merge(confusion_count_df, on=["Tag", "Model", "Metric"])
        return confusion_data

    def visualize(self):
        confusion_df = self.prepare_data()
        print(confusion_df)
        
        fig = px.bar(confusion_df,
                    x="Tag", y="Scale", color="Metric",
                    facet_row="Model",
                    title="Breakdown of Confusion Matrix Components: by Entity Tag, Categorized by Model",
                    labels={"Scale": "Scaled Counts"},
                    barmode='group',
                    template="plotly_white",
                    facet_row_spacing=0.1,  # Adjusted spacing
                    text='Count'  # Display the actual Count on top of each bar
                    )
        
        fig.show()



class ConfusionHeatmap(Visualization):
    def prepare_data(self):
        matrices = []
        for model_name, data_content in self.data.items():
            token_matrix = pd.DataFrame(data_content.token_confusion_matrix['confusion_matrix']).T 
            token_matrix['Model'] = model_name
            matrices.append(pd.concat([
                token_matrix, 
            ]))
        matrix_df = pd.concat(matrices)
        matrix_df.reset_index(inplace=True)
        matrix_df.rename(columns={'index': 'Tag'}, inplace=True)
        return self.replace_mappings(matrix_df)
        

    def visualize(self):
        token_matrix = self.prepare_data()
        token_matrix = token_matrix[token_matrix['Tag'] !='O']
        confusion_df = token_matrix.melt(
            id_vars=['Tag', 'Model'], 
            value_vars=['TP', 'FP', 'FN'], 
            var_name='Metric', 
            value_name='Count'
        )
        
        unique_models = confusion_df['Model'].unique()  # Only two models
        max_value = confusion_df['Count'].max()  # Get the max value for consistent color scaling

        # Create a subplot for each model
        fig = make_subplots(
            rows=len(unique_models), cols=1,  # Rows for models, single column
            subplot_titles=[f"{model}" for model in unique_models],
            shared_yaxes=True, horizontal_spacing=0.5, vertical_spacing=0.1
        )

        for idx, model in enumerate(unique_models):
            # Filter data for the current model
            filtered_data = confusion_df[confusion_df['Model'] == model]
            
            # Prepare heatmap data
            heatmap_data = filtered_data.pivot_table(index='Metric', columns='Tag', values='Count', fill_value=0)
            text_data = heatmap_data.astype(int)  # Convert values to integers for display

            # Add heatmap trace for the current model
            fig.add_trace(
                go.Heatmap(
                    z=heatmap_data.values,  # Heatmap values
                    x=heatmap_data.columns,  # Columns (Tags)
                    y=heatmap_data.index,  # Rows (Metrics)
                    colorscale='RdBu_r',
                    coloraxis="coloraxis",  # Unified color axis
                    text=text_data,  # Add text annotations
                    texttemplate="%{text}",  # Use the text values directly
                    hovertemplate="Metric: %{y}<br>Tag: %{x}<br>Count: %{text}<extra></extra>",
                ),
                row=idx + 1, col=1  # Stack rows for models
            )
        
        # Update layout with unified color axis and title
        fig.update_layout(
            coloraxis=dict(colorscale='RdBu_r', cmin=0, cmax=max_value, colorbar=dict(title="Counts")),
            title_text="Confusion Matrix Heatmap Categorized by Model",
            template="plotly_white",
            
            height=300 * len(unique_models),  # Adjust height dynamically based on the number of models
            width=800,  # Adjust the width if necessary
        )

        # Show the figure
        fig.show()
        

class ErrorTypeHeatmap(Visualization):
    def prepare_data(self):
        matrix_data = []
        # Step 1: Collect general error data
        for model_name, data_content in self.data.items():
            token_confusion = data_content.token_misclassifications 
                

            # Annotate with Model and Scheme
            token_confusion['Model'] = model_name
            token_confusion = self.replace_columns_and_index(token_confusion)
            matrix_data.append(token_confusion)
        
        # Step 2: Combine and process data
        matrix_df = pd.concat(matrix_data)
        matrix_df.reset_index(inplace=True)
        matrix_df.rename(columns={'index': 'Tag'}, inplace=True)
        matrix_df = self.replace_model(matrix_df)
        return matrix_df

    def visualize(self):
        token_misclassifications = self.prepare_data()
        
        # Group by 'Model'
        models = token_misclassifications['Model'].unique()

        # Create subplots for heatmaps
        fig = make_subplots(
            rows=1,
            cols=len(models),
            subplot_titles=models,
            horizontal_spacing=0.2
        )
        max_value = token_misclassifications.drop(columns=['Tag', 'Model']).values.max()


        # Iterate over each model and create a heatmap
        for i, model in enumerate(models, start=1):
            model_df = token_misclassifications[token_misclassifications['Model'] == model].set_index('Tag')
            model_df = model_df.drop(columns='Model')  # Drop the Model column
            fig.add_trace(
                go.Heatmap(
                    z=model_df.values,
                    x=model_df.columns,
                    y=model_df.index,
                    colorscale='RdBu_r',
                    coloraxis="coloraxis",  # Unified color axis
                    text=model_df.values,
                    texttemplate="%{text}",
                    hovertemplate="Metric: %{y}<br>Tag: %{x}<br>Count: %{text}<extra></extra>",
                ),
                row=1, col=i
            )

        # Update layout
        fig.update_layout(
            coloraxis=dict(colorscale='RdBu_r', cmin=0, cmax=max_value, colorbar=dict(title="Counts")),
            title="Errors Heatmap Across Entity Tag, Categorized by Model",
            template="plotly_white",
            width=1200,
            height=600,
            # margin=dict(t=50, l=50, r=50, b=50)
        )

        fig.show()
    
    
    
    def visualize_table(self):
        """
        Generates tables showing raw counts and percentages for each error type, 
        categorized by Scheme and Model.
        """
        # Step 1: Prepare the data
        token_misclassifications = self.prepare_data()
        print(token_misclassifications.to_string(index=False)) 
        


In [37]:
CONFIG_PATH = Path("/Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/dashboard-config.yaml")
config_manager = DashboardConfigManager(CONFIG_PATH)
dev_config = config_manager.development_config    

app = Dash(__name__, suppress_callback_exceptions=True)

app_config = config_manager.app_config
server = app.server  # Flask server instance for caching
variants_data = None

data_manager = DataManager(config_manager, server)
dash_data = data_manager.load_data()

2024-12-28 03:09:50 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/deformer-dashboard/notebooks/analysis/My Drive
2024-12-28 03:09:50 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2024-12-28 03:09:50 - INFO - Loading Dashboard Data from  /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/ANERCorp_CamelLab_arabertv02


  0%|          | 0/18 [00:00<?, ?it/s]

2024-12-28 03:09:56 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/deformer-dashboard/notebooks/analysis/My Drive
2024-12-28 03:09:56 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2024-12-28 03:09:56 - INFO - Loading Dashboard Data from  /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/conll2003_bert


  0%|          | 0/18 [00:00<?, ?it/s]

In [38]:
tr_df = dash_data['ANERCorp_CamelLab_arabertv02'].train_data

In [5]:
report_data = []
for data_name, data_content in dash_data.items():
	token_report = data_content.token_report
	token_report['Model'] = data_name
	report_data.append(pd.concat([
		token_report, 
	]))
report_df = pd.concat(report_data)
report_df = report_df[~report_df['Tag'].isin(['accuracy/micro', 'macro', 'weighted'])]    
tag_mapping = {
    'B-PERS': 'B-PER',
    'I-PERS': 'I-PER'
}

dataset_mapping = {
    'ANERCorp_CamelLab_arabertv02': 'AraBERTv02',
    'conll2003_bert': 'BERT'
}

report_df['Tag'] = report_df['Tag'].replace(tag_mapping)
report_df['Model'] = report_df['Model'].replace(dataset_mapping)
report_df


,Tag,Precision,Recall,F1,Support,Model
0,B-LOC,0.9056,0.9476,0.9261,668,AraBERTv02
1,B-MISC,0.8238,0.6766,0.7430,235,AraBERTv02
2,B-ORG,0.8237,0.7889,0.8059,450,AraBERTv02
3,B-PER,0.8955,0.8788,0.8871,858,AraBERTv02
4,I-LOC,0.8095,0.8193,0.8144,83,AraBERTv02
5,I-MISC,0.8193,0.4121,0.5484,165,AraBERTv02
6,I-ORG,0.8024,0.7236,0.7610,275,AraBERTv02
7,I-PER,0.9188,0.9002,0.9094,641,AraBERTv02
8,O,0.9857,0.9933,0.9895,21616,AraBERTv02
0,B-LOC,0.9287,0.9371,0.9329,1668,BERT


In [6]:
tr_support = tr_df[tr_df['Labels']!= -100]['True Labels'].value_counts().reset_index().rename(columns = {'True Labels': 'Tag', 'count': 'Train Support'})
tr = report_df.merge(tr_support, on='Tag')
tr

,Tag,Precision,Recall,F1,Support,Model,Train Support
0,B-LOC,0.9056,0.9476,0.9261,668,AraBERTv02,3772
1,B-MISC,0.8238,0.6766,0.7430,235,AraBERTv02,887
2,B-ORG,0.8237,0.7889,0.8059,450,AraBERTv02,1576
3,I-LOC,0.8095,0.8193,0.8144,83,AraBERTv02,525
4,I-MISC,0.8193,0.4121,0.5484,165,AraBERTv02,375
5,I-ORG,0.8024,0.7236,0.7610,275,AraBERTv02,1115
6,O,0.9857,0.9933,0.9895,21616,AraBERTv02,111488
7,B-LOC,0.9287,0.9371,0.9329,1668,BERT,3772
8,B-MISC,0.8296,0.8390,0.8343,702,BERT,887
9,B-ORG,0.9082,0.9169,0.9125,1661,BERT,1576


In [47]:
# Confusion matrix heatmap example
report_bar = ReportBarChart(dash_data, mappings)
report_bar.visualize_f1()

In [50]:
# Confusion matrix heatmap example
report_bar = ReportBarChart(dash_data, mappings)
report_bar.visualize()

In [51]:
# Confusion matrix heatmap example
report_bar = ReportBarChart(dash_data, mappings)
report_bar.visualize_support()

In [52]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Prepare Train and Test correlation data
metrics = ["Precision", "Recall"]
train_correlation_data = []
test_correlation_data = []
data = tr[tr["Tag"] != "O"]

for model in data["Model"].unique():
    model_data = data[data["Model"] == model]
    for metric in metrics:
        test_correlation = model_data[["Support", metric]].corr().iloc[0, 1]
        train_correlation = model_data[["Train Support", metric]].corr().iloc[0, 1]
        test_correlation_data.append({"Metric": metric, "Model": model, "Correlation": test_correlation})
        train_correlation_data.append({"Metric": metric, "Model": model, "Correlation": train_correlation})

# Create DataFrames for Train and Test
test_correlation_df = pd.DataFrame(test_correlation_data)
test_correlation_df['Correlation'] = test_correlation_df['Correlation'].round(3)
train_correlation_df = pd.DataFrame(train_correlation_data)
train_correlation_df['Correlation'] = train_correlation_df['Correlation'].round(3)

# Pivot for heatmap format
test_heatmap = test_correlation_df.pivot(index="Metric", columns="Model", values="Correlation")
train_heatmap = train_correlation_df.pivot(index="Metric", columns="Model", values="Correlation")

# Create subplots for Train and Test
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Train Correlation", "Test Correlation"),
    horizontal_spacing=0.2,
)

# Add Train heatmap with shared color axis
fig.add_trace(
    go.Heatmap(
        z=train_heatmap.values,
        x=train_heatmap.columns,
        y=train_heatmap.index,
        text=train_heatmap.values,
        texttemplate="%{text}",
        hovertemplate="Metric: %{y}<br>Tag: %{x}<br>Count: %{text}<extra></extra>",
        colorscale="RdBu_r",
        zmin=-1,
        zmax=1,
        coloraxis="coloraxis",  # Shared color axis
    ),
    row=1,
    col=1,
)

# Add Test heatmap with shared color axis
fig.add_trace(
    go.Heatmap(
        z=test_heatmap.values,
        x=test_heatmap.columns,
        y=test_heatmap.index,
        text=test_heatmap.values,
        texttemplate="%{text}",
        hovertemplate="Metric: %{y}<br>Tag: %{x}<br>Count: %{text}<extra></extra>",
        colorscale="RdBu_r",
        zmin=-1,
        zmax=1,
        coloraxis="coloraxis",  # Shared color axis
    ),
    row=1,
    col=2,
)

# Update layout with a single shared color axis
fig.update_layout(
    coloraxis=dict(colorscale="RdBu_r", cmin=-1, cmax=1, colorbar=dict(title="Correlation")),
    title="Train vs Test Correlation Heatmaps Across Metrics and Models",
    width=1000,
    height=500,
    template="plotly_white",
    margin=dict(t=50, l=50, r=50, b=50),
)

fig.show()


In [26]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Prepare Train and Test correlation data
metrics = ["Precision", "Recall"]
train_correlation_data = []
test_correlation_data = []
data = tr[tr["Tag"] != "O"]

for model in data["Model"].unique():
    model_data = data[data["Model"] == model]
    for metric in metrics:
        test_correlation = model_data[["Support", metric]].corr('spearman').iloc[0, 1]
        train_correlation = model_data[["Train Support", metric]].corr('spearman').iloc[0, 1]
        test_correlation_data.append({"Metric": metric, "Model": model, "Correlation": test_correlation})
        train_correlation_data.append({"Metric": metric, "Model": model, "Correlation": train_correlation})

# Create DataFrames for Train and Test
test_correlation_df = pd.DataFrame(test_correlation_data)
test_correlation_df['Correlation'] = test_correlation_df['Correlation'].round(3)
train_correlation_df = pd.DataFrame(train_correlation_data)
train_correlation_df['Correlation'] = train_correlation_df['Correlation'].round(3)

# Pivot for heatmap format
test_heatmap = test_correlation_df.pivot(index="Metric", columns="Model", values="Correlation")
train_heatmap = train_correlation_df.pivot(index="Metric", columns="Model", values="Correlation")

# Create subplots for Train and Test
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Train Correlation", "Test Correlation"),
    horizontal_spacing=0.2,
)

# Add Train heatmap with shared color axis
fig.add_trace(
    go.Heatmap(
        z=train_heatmap.values,
        x=train_heatmap.columns,
        y=train_heatmap.index,
        text=train_heatmap.values,
        texttemplate="%{text}",
        hovertemplate="Metric: %{y}<br>Tag: %{x}<br>Count: %{text}<extra></extra>",
        colorscale="RdBu_r",
        zmin=-1,
        zmax=1,
        coloraxis="coloraxis",  # Shared color axis
    ),
    row=1,
    col=1,
)

# Add Test heatmap with shared color axis
fig.add_trace(
    go.Heatmap(
        z=test_heatmap.values,
        x=test_heatmap.columns,
        y=test_heatmap.index,
        text=test_heatmap.values,
        texttemplate="%{text}",
        hovertemplate="Metric: %{y}<br>Tag: %{x}<br>Count: %{text}<extra></extra>",
        colorscale="RdBu_r",
        zmin=-1,
        zmax=1,
        coloraxis="coloraxis",  # Shared color axis
    ),
    row=1,
    col=2,
)

# Update layout with a single shared color axis
fig.update_layout(
    coloraxis=dict(colorscale="RdBu_r", cmin=-1, cmax=1, colorbar=dict(title="Correlation")),
    title="Train vs Test Correlation Heatmaps Across Metrics and Models",
    width=1000,
    height=500,
    template="plotly_white",
    margin=dict(t=50, l=50, r=50, b=50),
)

fig.show()


In [27]:
import pandas as pd
import plotly.express as px

# Filter out the 'O' tag
data = tr[tr["Tag"] != "O"]

# Melt the DataFrame for plotting
melted_df = data.melt(
    id_vars=["Tag", "Model", "Train Support", "Support"],
    value_vars=["Precision", "Recall"],
    var_name="Metric",
    value_name="Value"
)

# Create separate columns for Train and Test support
melted_train = melted_df.copy()
melted_train["Support Type"] = "Train"
melted_train["Support Value"] = melted_train["Train Support"]

melted_test = melted_df.copy()
melted_test["Support Type"] = "Test"
melted_test["Support Value"] = melted_test["Support"]

# Combine Train and Test data
combined_df = pd.concat([melted_train, melted_test])

# Scatter plot with Train and Test as colors
fig = px.scatter(
    combined_df,
    x="Support Value",
    y="Value",
    color="Support Type",  # Color by Train or Test
    facet_col="Metric",
    facet_row="Model",
    facet_row_spacing=0.1,  # Adjusted spacing
    facet_col_spacing=0.03,
    hover_data=["Tag"],
    title="Scatter Plot of Train and Test Support vs Metrics Across Models",
    labels={
        "Support Value": "Support",
        "Value": "Metric Value",
        "Metric": "Metric",
        "Support Type": "Support Type",
    },
    template="plotly_white"
)

# Update layout for better readability
fig.update_layout(
    width=1000,
    height=600,
    xaxis_title="Support",
    yaxis_title="Metric Value",
    legend_title="Support Type",
    # margin=dict(t=50, l=50, r=50, b=50)
)

fig.show()


In [41]:
confusion_bar = ConfusionBarChart(dash_data, mappings)
confusion_bar.visualize()

       Tag       Model Metric     Scale  Count
0   B-MISC  AraBERTv02     TP  0.591078    159
1   I-MISC  AraBERTv02     TP  0.377778     68
2    B-ORG  AraBERTv02     TP  0.674905    355
3        O  AraBERTv02     TP  0.979249  21472
4    I-LOC  AraBERTv02     TP  0.686869     68
5    B-LOC  AraBERTv02     TP  0.862398    633
6    I-ORG  AraBERTv02     TP  0.614198    199
7    B-PER  AraBERTv02     TP  0.797040    754
8    I-PER  AraBERTv02     TP  0.833815    577
9   B-MISC        BERT     TP  0.715674    589
10  I-MISC        BERT     TP  0.557093    161
11   B-ORG        BERT     TP  0.839118   1523
12   B-PER        BERT     TP  0.933573   1560
13       O        BERT     TP  0.990688  38088
14   I-LOC        BERT     TP  0.792642    237
15   B-LOC        BERT     TP  0.874161   1563
16   I-ORG        BERT     TP  0.840000    777
17   I-PER        BERT     TP  0.977797   1145
18  B-MISC  AraBERTv02     FP  0.126394     34
19  I-MISC  AraBERTv02     FP  0.083333     15
20   B-ORG  A

In [47]:
# Confusion matrix heatmap example
confusion_heatmap = ConfusionHeatmap(dash_data, mappings)
confusion_heatmap.visualize()

In [103]:
error_type_heatmap = ErrorTypeHeatmap(dash_data, mappings)
error_type_heatmap.visualize()

In [96]:
error_type_heatmap.visualize_table()


   Tag  B-MISC  I-MISC  B-ORG  O  I-LOC  B-LOC  I-ORG  B-PER  I-PER      Model
B-MISC       0       1     10 57      0      4      0      0      4 AraBERTv02
I-MISC       5       0      2 59      3     12     14      1      1 AraBERTv02
 B-ORG       7       0      0 49      4     19      1     15      0 AraBERTv02
     O      14      10     39  0      2     14     25     34      6 AraBERTv02
 I-LOC       0       0      0  8      0      7      0      0      0 AraBERTv02
 B-LOC       3       0      3 24      1      0      2      1      1 AraBERTv02
 I-ORG       0       2      9 50      5      2      0      2      6 AraBERTv02
 B-PER       4       1     12 44      0      8      2      0     33 AraBERTv02
 I-PER       1       1      1 20      1      0      5     35      0 AraBERTv02
B-MISC       0      11     39 30      0     25      1      7      0       BERT
I-MISC       5       0      1 21      8      0     17      2      1       BERT
 B-ORG      34       0      0 21      0     57     1

In [23]:
df = dash_data['ANERCorp_CamelLab_arabertv02'].analysis_data
df[(df['True Labels'] != 'B-LOC')&(df['Pred Labels'] == 'B-LOC')]['True Labels'].value_counts()
# df[(df['Pred Labels'] == 'B-LOC') & (df['True Labels'] == 'B-MISC')]

True Labels
B-ORG     19
O         14
I-MISC    12
B-PERS     8
I-LOC      7
B-MISC     4
I-ORG      2
Name: count, dtype: int64

In [22]:
dash_data['ANERCorp_CamelLab_arabertv02'].token_misclassifications

,B-MISC,I-MISC,B-ORG,O,I-LOC,B-LOC,I-ORG,B-PERS,I-PERS
B-MISC,0,1,10,57,0,4,0,0,4
I-MISC,5,0,2,59,3,12,14,1,1
B-ORG,7,0,0,49,4,19,1,15,0
O,14,10,39,0,2,14,25,34,6
I-LOC,0,0,0,8,0,7,0,0,0
B-LOC,3,0,3,24,1,0,2,1,1
I-ORG,0,2,9,50,5,2,0,2,6
B-PERS,4,1,12,44,0,8,2,0,33
I-PERS,1,1,1,20,1,0,5,35,0
